# Land-use forcing

Smith et al. 2020:

- 2014 - 1850 land use forcing of -0.18 W m-2

The time series that scales with AFOLU is our best bet.

In [ ]:
import os

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties
from scipy.optimize import root

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import pooch

In [ ]:
climate_response_df = pd.read_csv('../data/fair-calibrations/4xCO2_energy_balance_ebm3.csv')

In [ ]:
target_forcing = -0.18

scenarios = ['ssp126', 'ssp245', 'ssp370', 'ssp585']
configs = ['UKESM1-0-LL']
species = ['CO2 AFOLU', 'Land use']

In [ ]:
def erf_rootfinder(x):
    f = FAIR()
    f.define_time(1850, 2101, 1)
    f.define_scenarios(scenarios)
    f.define_configs(configs)
    f.define_species(*read_properties(species=species, filename='../data/species_configs_properties_vanilla.csv'))
    f.allocate()
    f.fill_from_rcmip()
#    f.fill_species_configs()
    fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
    fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
    fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
    fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])
    fill(f.species_configs['land_use_cumulative_emissions_to_forcing'], x, specie='CO2 AFOLU')
    initialise(f.forcing, 0)
    initialise(f.temperature, 0)
    initialise(f.cumulative_emissions, 0)  # necessary
    f.run(progress=False)
    return f.forcing[165, 0, 0, 1] - target_forcing

In [ ]:
sol = root(erf_rootfinder, -0.006)

In [ ]:
sol.x[0]

In [ ]:
f = FAIR()
f.define_time(1850, 2101, 1)
f.define_scenarios(scenarios)
f.define_configs(configs)
f.define_species(*read_properties(species=species, filename='../data/species_configs_properties_vanilla.csv'))
f.allocate()
f.fill_from_rcmip()
#    f.fill_species_configs()
fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])
fill(f.species_configs['land_use_cumulative_emissions_to_forcing'], sol.x[0], specie='CO2 AFOLU')
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)  # necessary
f.run()

In [ ]:
f.forcing[:,:,0,-1]

In [ ]:
os.makedirs('../data/calibration/', exist_ok=True)

In [ ]:
landuse_erf = f.forcing[:,:,0,-1].to_pandas()
# landuse_erf.name = 'landuse_erf'
landuse_erf.to_csv('../data/calibration/landuse_forcing.csv')

In [ ]:
# landuse_erf